In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [31]:
specs = pd.read_csv('/code/data/raw/specs.csv')
train_df = pd.read_csv('/code/data/raw/train.csv')
test_df = pd.read_csv('/code/data/raw/test.csv')

In [3]:
specs.head()

,event_id,info,args
0,2b9272f4,The end of system-initiated feedback (Correct)...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
1,df4fe8b6,The end of system-initiated feedback (Incorrec...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
2,3babcb9b,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
3,7f0836bf,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
4,ab3136ba,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."


In [4]:
pd.get_dummies(specs.args)

,"[{""name"":""bucket"",""type"":""int"",""info"":""capacity of the bucket clicked""},{""name"":""buckets_placed"",""type"":""array->int"",""info"":""list of buckets placed under cauldrons from left to right""},{""name"":""target_bucket"",""type"":""int"",""info"":""capacity of the correct bucket""},{""name"":""coordinates"",""type"":""object"",""info"":""the game screen coordinates of the click\r\ndictionary:\r\n{“x”: integer, “y”: integer, “stage_width”: integer, “stage_height”: integer}\r\n""},{""name"":""correct"",""type"":""boolean"",""info"":""is this the correct bucket?""},{""name"":""prompt"",""type"":""string"",""info"":""“bucket that holds the most”, “holds least”, etc. – the prompt given to the player""},{""name"":""game_time"",""type"":""int"",""info"":""millisecond count since start of game""},{""name"":""event_count"",""type"":""int"",""info"":""session event counter""},{""name"":""event_code"",""type"":""int"",""info"":""event class identifier""}]","[{""name"":""event_count"",""type"":""int"",""info"":""session event counter""},{""name"":""event_code"",""type"":""int"",""info"":""event class identifier""}]","[{""name"":""game_time"",""type"":""int"",""info"":""millisecond count since start of game""},{""name"":""bucket"",""type"":""int"",""info"":""capacity of the bucket being dragged""},{""name"":""buckets"",""type"":""array->int"",""info"":""list of buckets available in the resource area with 0 for missing buckets""},{""name"":""coordinates"",""type"":""object"",""info"":""the game screen coordinates of the placement\ndictionary:\n{“x”: integer, “y”: integer, “stage_width”: integer, “stage_height”: integer}""},{""name"":""buckets_placed"",""type"":""array->int"",""info"":""list of buckets placed under cauldrons from left to right with 0 for missing buckets""},{""name"":""source"",""type"":""string"",""info"":""“left”, “middle”, “right”, or “resources” – location the bucket was dragged from""},{""name"":""event_count"",""type"":""int"",""info"":""session event counter""},{""name"":""event_code"",""type"":""int"",""info"":""event class identifier""}]","[{""name"":""game_time"",""type"":""int"",""info"":""millisecond count since start of game""},{""name"":""bucket"",""type"":""int"",""info"":""capacity of the bucket being dragged""},{""name"":""coordinates"",""type"":""object"",""info"":""the game screen coordinates of the placement\ndictionary:\n{“x”: integer, “y”: integer, “stage_width”: integer, “stage_height”: integer}""},{""name"":""buckets"",""type"":""array->int"",""info"":""list of buckets available in the resource area with 0 for missing buckets""},{""name"":""buckets_placed"",""type"":""array->int"",""info"":""list of buckets placed under cauldrons from left to right with 0 for missing buckets""},{""name"":""source"",""type"":""string"",""info"":""“left”, “middle”, or “right” – location the bucket was dragged from""},{""name"":""duration"",""type"":""int"",""info"":""the duration of the drag in milliseconds""},{""name"":""event_count"",""type"":""int"",""info"":""session event counter""},{""name"":""event_code"",""type"":""int"",""info"":""event class identifier""}]","[{""name"":""game_time"",""type"":""int"",""info"":""millisecond count since start of game""},{""name"":""bucket"",""type"":""int"",""info"":""capacity of the bucket placed""},{""name"":""destination"",""type"":""string"",""info"":""“left”, “middle”, or “right” – position where the cauldron was placed""},{""name"":""coordinates"",""type"":""object"",""info"":""the game screen coordinates of the placement\ndictionary:\n{“x”: integer, “y”: integer, “stage_width”: integer, “stage_height”: integer}""},{""name"":""cauldron"",""type"":""int"",""info"":""capacity of the cauldron where the bucket was placed""},{""name"":""buckets"",""type"":""array->int"",""info"":""list of buckets available in the resource area with 0 for missing buckets""},{""name"":""buckets_placed"",""type"":""array->int"",""info"":""list of buckets placed under cauldrons from

In [5]:
len(specs)

386

In [6]:
vectorizer = TfidfVectorizer()

In [7]:
X = vectorizer.fit_transform(specs['info'].values)

In [8]:
X.shape

(386, 443)

In [9]:
vectorizer.get_feature_names()[0:10]

['3010',
 '3020',
 '3021',
 'about',
 'acceptable',
 'accuracy',
 'action',
 'activity',
 'actually',
 'after']

In [10]:
X.toarray()

array([[0.        , 0.        , 0.        , ..., 0.13042985, 0.20638076,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.11247416,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.19987916, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [11]:
clusters = KMeans(n_clusters=20, random_state=0).fit_predict(X.toarray())

In [12]:
clusters = ['event_' + str(i) for i in clusters]

In [13]:
specs['cluster'] = clusters

In [14]:
cluster_df = specs.set_index('event_id')['cluster']

In [15]:
cluster_df

event_id
2b9272f4    event_15
df4fe8b6     event_5
3babcb9b     event_0
7f0836bf     event_0
ab3136ba     event_0
              ...   
29f54413    event_16
06372577    event_16
2a444e03    event_12
9e6b7fb5    event_14
d3640339     event_3
Name: cluster, Length: 386, dtype: object

In [16]:
test = pd.read_csv('/code/data/raw/test.csv')

In [17]:
test.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,0ea9ecc81a565215,2019-09-10T16:50:24.910Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,c1ea43d8b8261d27,2019-09-10T16:50:55.503Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,27253bdc,7ed86c6b72e725e2,2019-09-10T16:51:51.805Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK
3,27253bdc,7e516ace50e7fe67,2019-09-10T16:53:12.825Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Crystal Caves - Level 1,Clip,CRYSTALCAVES
4,7d093bf9,a022c3f60ba547e7,2019-09-10T16:54:12.115Z,"{""version"":""1.0"",""round"":0,""event_count"":1,""ga...",00abaee7,1,2000,0,Chow Time,Game,CRYSTALCAVES


In [28]:
new_test = pd.merge(test, cluster_df,  left_on='event_id', right_index=True, indicator=True)

In [29]:
new_test.drop('event_id', axis=1, inplace=True)

In [30]:
new_test.head()

,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world,cluster,_merge
0,0ea9ecc81a565215,2019-09-10T16:50:24.910Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE,event_12,both
1,c1ea43d8b8261d27,2019-09-10T16:50:55.503Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK,event_12,both
2,7ed86c6b72e725e2,2019-09-10T16:51:51.805Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK,event_12,both
3,7e516ace50e7fe67,2019-09-10T16:53:12.825Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Crystal Caves - Level 1,Clip,CRYSTALCAVES,event_12,both
163,1c203986674d7d9b,2019-09-10T16:56:39.003Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Balancing Act,Clip,CRYSTALCAVES,event_12,both


In [21]:
new_test = new_test.rename(columns={'cluster': 'event_id'})

In [24]:
new_test

,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world,event_id
0,0ea9ecc81a565215,2019-09-10T16:50:24.910Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE,event_12
1,c1ea43d8b8261d27,2019-09-10T16:50:55.503Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK,event_12
2,7ed86c6b72e725e2,2019-09-10T16:51:51.805Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK,event_12
3,7e516ace50e7fe67,2019-09-10T16:53:12.825Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Crystal Caves - Level 1,Clip,CRYSTALCAVES,event_12
163,1c203986674d7d9b,2019-09-10T16:56:39.003Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Balancing Act,Clip,CRYSTALCAVES,event_12
...,...,...,...,...,...,...,...,...,...,...,...
856665,0cea1e022bf35e98,2019-10-04T15:20:45.160Z,"{""dwell_time"":1937,""object"":""bottle_5"",""round""...",bbd98f65,30,4080,48746,Bottle Filler (Activity),Activity,MAGMAPEAK,event_2
856670,0cea1e022bf35e98,2019-10-04T15:20:57.657Z,"{""dwell_time"":2725,""object"":""bottle_11"",""round...",bbd98f65,35,4080,61236,Bottle Filler (Activity),Activity,MAGMAPEAK,event_2
725120,53d15c3e3edafca9,2019-08-11T18:10:23.085Z,"{""dwell_time"":5800,""object"":""House5"",""round"":1...",9aa5dcea,22,4080,24185,All Star Sorting,Game,TREETOPCITY,event_2
809381,b53eb30280d4c181,2019-09-25T16:08:04.065Z,"{""dwell_time"":54553,""object"":""House5"",""round"":...",ad0f90e1,115,4080,137110,All Star Sorting,Game,TREETOPCITY,event_2


In [23]:
new_test.reset_index()

,index,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world,event_id
0,0,0ea9ecc81a565215,2019-09-10T16:50:24.910Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE,event_12
1,1,c1ea43d8b8261d27,2019-09-10T16:50:55.503Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK,event_12
2,2,7ed86c6b72e725e2,2019-09-10T16:51:51.805Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK,event_12
3,3,7e516ace50e7fe67,2019-09-10T16:53:12.825Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Crystal Caves - Level 1,Clip,CRYSTALCAVES,event_12
4,163,1c203986674d7d9b,2019-09-10T16:56:39.003Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Balancing Act,Clip,CRYSTALCAVES,event_12
...,...,...,...,...,...,...,...,...,...,...,...,...
1156409,856665,0cea1e022bf35e98,2019-10-04T15:20:45.160Z,"{""dwell_time"":1937,""object"":""bottle_5"",""round""...",bbd98f65,30,4080,48746,Bottle Filler (Activity),Activity,MAGMAPEAK,event_2
1156410,856670,0cea1e022bf35e98,2019-10-04T15:20:57.657Z,"{""dwell_time"":2725,""object"":""bottle_11"",""round...",bbd98f65,35,4080,61236,Bottle Filler (Activity),Activity,MAGMAPEAK,event_2
1156411,725120,53d15c3e3edafca9,2019-08-11T18:10:23.085Z,"{""dwell_time"":5800,""object"":""House5"",""round"":1...",9aa5dcea,22,4080,24185,All Star Sorting,Game,TREETOPCITY,event_2
1156412,809381,b53eb30280d4c181,2019-09-25T16:08:04.065Z,"{""dwell_time"":54553,""object"":""House5"",""round"":...",ad0f90e1,115,4080,137110,All Star Sorting,Game,TREETOPCITY,event_2


In [25]:
vectorizer = TfidfVectorizer()

In [26]:
X = vectorizer.fit_transform(specs['args'].values)

In [27]:
vectorizer.get_feature_names()[0:10]


['1sec',
 'about',
 'activity',
 'add',
 'added',
 'advance',
 'after',
 'already',
 'amount',
 'an']

In [30]:
X.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])